In [2]:
import sys
from os import system
import os
import pandas as pd
import numpy as np
import time
import copy
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score
from sklearn.externals.joblib import dump, load

# sys.path.insert(1, os.path.join(sys.path[0], 'datagen/img_generator/'))
# from network import Net, Net_Regressor

In [33]:
data = pd.read_csv('classifier_files/data.csv')
print "Total length of data: ", data.shape
data[0:10]

Total length of data:  (20192, 19)


,true_init_x,true_init_y,true_init_z,blur_coeff,var_sum,diff_x,diff_y,diff_z,diff_phi,diff_theta,diff_psi,r_std,phi_std,theta_std,psi_std,Tf,MP_Method,Cost,Status
0,-0.193083,-0.085577,-2.073306,0.0,0.062777,0.173774,-3.445764,0.050844,-0.021509,0.072953,0.715764,0.028558,0.002208,0.006967,0.025043,2.333994,min_vel,439.659511,SUCCESS
1,-0.168562,-0.561884,-2.072260,0.0,0.019096,0.449073,-3.754241,-0.470847,-0.021509,0.072953,0.065251,0.000000,0.000098,0.008652,0.010345,2.566871,min_vel,238.153363,SUCCESS
2,-0.044489,-1.730302,-2.125663,0.0,0.363184,-0.225235,-1.443765,0.094266,-0.021509,0.072953,0.201928,0.323641,0.009191,0.002800,0.027552,1.048021,min_vel,30.032970,SUCCESS
3,-0.011622,-2.175070,-2.154150,0.0,0.185138,-0.173911,-1.663455,-0.284998,-0.021509,0.072953,0.500841,0.152674,0.006536,0.001376,0.024552,1.198461,min_vel,51.768222,SUCCESS
4,-0.072312,-2.607443,-2.184460,0.0,0.450476,0.128647,-1.745246,-0.002598,-0.021509,0.072953,0.132464,0.419632,0.006863,0.000000,0.023980,1.233005,min_vel,60.806281,SUCCESS
5,-0.096470,-3.064906,-2.216762,0.0,0.249108,1.072036,-3.634816,0.232052,-0.021509,0.072953,0.035242,0.228495,0.000000,0.001183,0.019430,2.558131,min_vel,227.679519,SUCCESS
6,0.194677,-4.211246,-2.216990,0.0,0.023317,-0.032028,-0.983332,0.429236,-0.021509,0.072953,0.105037,0.000000,0.003570,0.001434,0.018312,0.794966,min_vel,16.964613,SUCCESS
7,0.291833,-4.537059,-2.192402,0.0,0.020371,0.019956,-0.993838,0.062526,-0.021509,0.072953,0.047952,0.000000,0.005209,0.008161,0.007001,0.744850,min_vel,24.577205,SUCCESS
8,0.311979,-4.783995,-2.160646,0.0,0.130170,-0.084228,-0.818867,0.237791,-0.021509,0.072953,0.779829,0.078786,0.010415,0.013663,0.027306,0.654753,min_vel,16.368391,SUCCESS
9,0.306588,-4.987317,-2.131710,0.0,1.553038,1.376332,-2.729421,-0.228199,-0.021509,0.072953,0.246488,1.471256,0.010279,0.000000,0.071503,2.084594,min_vel,201.222737,SUCCESS


### Classify the data points into lists considering the initial positions

In [34]:
eps = 0.1 # if the difference is as much as eps, then classify them into the same list
N_list = 1500 # number of lists
data_list = {}
data_list_no_failures = {} 
for i in range(N_list):
    list_name = "list_" + str(i+1)
    data_list[list_name] = []
    data_list_no_failures[list_name] = []
    
data_arr = data.values
N_array = data_arr.shape[0]
duplicate_entries = 0
for i in range(N_array):
    element = data_arr[i,:]
    x1,y1,z1,cost1 = element[0], element[1], element[2], element[17] 
    
    appending_complete = False
    for j in range(N_list):
        
        if appending_complete:
            break
        
        list_name = "list_" + str(j+1)
        if data_list[list_name]:
            for k in range(len(data_list[list_name])):
                element_compare = data_list[list_name][k]
                x2,y2,z2,cost2 = element_compare[0], element_compare[1], element_compare[2], element_compare[17]
                
                # if the initial positions of data points are in eps range, then classify them into the same list
                if (x2 - eps <= x1 <= x2 + eps) and (y2 - eps <= y1 <= y2 + eps)  and (z2 - eps <= z1 <= z2 + eps):
                    # if the cost values of different data points are exactly same, this means a duplicate data entry
                    # in this case, do not add the data into the lists
                    appending_complete = True
                    if cost2 - eps <= cost1 <= cost2 + eps:
                        duplicate_entries += 1                        
                        break
                    data_list[list_name].append(list(element[0:-1]))
                    
                    if cost1 < 1e12:
                        data_list_no_failures[list_name].append(list(element[0:-1]))
                    break
        else:
            # if this data point is not near of any other data points, put it into a new list
            data_list[list_name].append(list(element[0:-1]))
#             print "No data in " + list_name + " x1-y1-z1 is added here." 
            appending_complete = True
    
print "Process is completed."
print "Number of duplicate entries: ", duplicate_entries

Process is completed.
Number of duplicate entries:  41


### How many data points are there in each list?

In [35]:
N_sum = 0
N_sum_no_failures = 0
for k in range(N_list):
    list_name = "list_" + str(k+1)
    N_sum += len(data_list[list_name])
    N_sum_no_failures += len(data_list_no_failures[list_name])
    
print "Total number of same entries: ", duplicate_entries
print "Total number of data points: ", len(data.values)
print "Total number of classified data points: ", N_sum
print "Total number of data points with no failures: ", N_sum_no_failures

Total number of same entries:  41
Total number of data points:  20192
Total number of classified data points:  16863
Total number of data points with no failures:  15177


### Sort the data points by their cost value

In [36]:
from operator import itemgetter
for k in range(N_list):
    list_name = "list_" + str(k+1)
    data_list[list_name] = sorted(data_list[list_name], key=itemgetter(17))
    data_list_no_failures[list_name] = sorted(data_list_no_failures[list_name], key=itemgetter(17))

### Let us take a glance at the lists, we filled up

In [37]:
for k in range(N_list):
    list_name = "list_" + str(k+1)
    if data_list_no_failures[list_name]:
        first = data_list_no_failures[list_name][0]
        last = data_list_no_failures[list_name][-1]
        print "\nlist_"+str(k+1) + " - Length: " + str(len(data_list_no_failures[list_name]))
        print "First element, x: {0:.4}, y: {1:.4}, z: {2:.4}, cost: {3:.6} ".format(first[0], first[1], first[2], first[17])
        print "Last element, x: {0:.4}, y: {1:.4}, z: {2:.4}, cost: {3:.6} ".format(last[0], last[1], last[2], last[17])


list_1 - Length: 1528
First element, x: -0.06063, y: -0.4587, z: -1.814, cost: 2.86665 
Last element, x: 0.2883, y: 0.02303, z: -2.003, cost: 71781.2 

list_2 - Length: 112
First element, x: -0.006688, y: -0.8276, z: -1.984, cost: 12.486 
Last element, x: 0.04232, y: -0.4501, z: -2.035, cost: 5178.06 

list_3 - Length: 400
First element, x: 0.2382, y: -1.877, z: -2.127, cost: 4.19462 
Last element, x: 0.05686, y: -1.332, z: -2.243, cost: 5076.89 

list_4 - Length: 347
First element, x: -0.02548, y: -2.485, z: -2.161, cost: 1.86328 
Last element, x: -0.03495, y: -2.067, z: -2.102, cost: 6287.5 

list_5 - Length: 211
First element, x: 0.09891, y: -2.646, z: -2.14, cost: 11.4343 
Last element, x: -0.0836, y: -2.28, z: -1.671, cost: 4536.29 

list_6 - Length: 300
First element, x: 0.1655, y: -3.356, z: -2.15, cost: 4.11645 
Last element, x: 0.02129, y: -3.171, z: -2.154, cost: 5179.54 

list_7 - Length: 785
First element, x: 0.17, y: -4.331, z: -2.192, cost: 0.740963 
Last element, x: 0.1

### We need to convert algorithm types into numbers

In [38]:
algorithm_dict = {"min_vel":0, "min_acc":1, "min_jerk":2, "min_jerk_full_stop":3}
algorithms = ["min_vel", "min_acc", "min_jerk", "min_jerk_full_stop"]

for k in range(N_list):
    list_name = "list_" + str(k+1)
    if data_list[list_name]:
        for j in range(len(data_list[list_name])):
            index = data_list[list_name][j][16]
            data_list[list_name][j][16] = int(algorithm_dict[index])
            
    if data_list_no_failures[list_name]:
        for j in range(len(data_list_no_failures[list_name])):
            index = data_list_no_failures[list_name][j][16]
            data_list_no_failures[list_name][j][16] = int(algorithm_dict[index])

### Details of cost values in the lists

In [40]:
cost_median = {}
for k in range(N_list):
    list_name = "list_" + str(k+1)
    if data_list[list_name]:
        arr = np.array(data_list[list_name],dtype=np.float)
        print "\nList: ", list_name
        print "Cost values: ", arr[:,17]
        print "Mean cost: ", np.mean(arr[:,17])
        print "Median cost: ", np.median(arr[:,17])
        print "Std cost: ", np.std(arr[:,17])
        
        cost_median[list_name] = np.median(arr[:,17])


List:  list_1
Cost values:  [2.86664978e+00 3.51342192e+00 4.20434368e+00 ... 7.17812415e+04
 1.00000000e+12 1.00000000e+12]
Mean cost:  1306339722.0999773
Median cost:  620.6393370254339
Std cost:  36119651227.72818

List:  list_2
Cost values:  [1.24859889e+01 2.00524033e+01 2.16579372e+01 2.31828043e+01
 2.36993577e+01 2.53088315e+01 2.77054674e+01 3.20456710e+01
 3.72882900e+01 4.10325060e+01 4.37501297e+01 4.85651942e+01
 5.12286525e+01 5.35097235e+01 5.40358385e+01 5.41669786e+01
 6.09349469e+01 6.19276818e+01 6.19992556e+01 6.46773587e+01
 6.70111802e+01 6.94406329e+01 6.98893199e+01 7.19886197e+01
 8.08034596e+01 8.12599976e+01 8.23616685e+01 8.24979884e+01
 8.60709455e+01 9.07029899e+01 9.19792469e+01 9.99508172e+01
 1.02871949e+02 1.03618198e+02 1.13772565e+02 1.17116005e+02
 1.19159452e+02 1.21073124e+02 1.22256343e+02 1.27506167e+02
 1.29198337e+02 1.35829932e+02 1.41309999e+02 1.41450726e+02
 1.42793860e+02 1.46551365e+02 1.47225612e+02 1.51867824e+02
 1.54211112e+02 1.576

Median cost:  131.22835300975066
Std cost:  319.1956100068478

List:  list_135
Cost values:  [  896.03221643  5459.25294265 67985.48798736]
Mean cost:  24780.257715478754
Median cost:  5459.252942648523
Std cost:  30607.45756602233

List:  list_136
Cost values:  [4550.53607923 7389.76919947]
Mean cost:  5970.152639353535
Median cost:  5970.152639353535
Std cost:  1419.6165601198031

List:  list_137
Cost values:  [ 26.65280259  35.33298155  84.59666991 268.20959503 283.20720307
 354.6992112  670.3565397 ]
Mean cost:  246.15071472231924
Median cost:  268.2095950296533
Std cost:  211.22645092840028

List:  list_138
Cost values:  [  29.3769506   107.01839913  111.05308124  114.50575785  275.16604171
  318.7324      419.21048708  434.62632752  441.49767826 1114.97090578]
Mean cost:  336.61580291690996
Median cost:  296.94922085244116
Std cost:  297.4361783125189

List:  list_139
Cost values:  [ 17.74341627  24.98701138  43.71137255  45.79676703 101.7658017
 140.73955347 155.66790177 180.514

Median cost:  98.43387749006223
Std cost:  372677996219.14526

List:  list_285
Cost values:  [  16.9493203   109.85929464  163.59335663  169.68953187  291.84822786
  299.2064612   308.84228644  320.60627635  703.05610138 1100.1315252
 3607.20287006]
Mean cost:  644.6350229029396
Median cost:  299.2064611989865
Std cost:  981.0095849950181

List:  list_286
Cost values:  [ 5.83326667 40.96892258]
Mean cost:  23.40109462763676
Median cost:  23.40109462763676
Std cost:  17.567827953033376

List:  list_287
Cost values:  [ 20.11492319  31.42736171  56.8869879  121.1582525 ]
Mean cost:  57.39688132730631
Median cost:  44.157174805628614
Std cost:  39.14758229144334

List:  list_288
Cost values:  [ 17.46013368  19.74080208  22.95409398  23.22222168  27.76824972
  38.20228925  39.85714504  40.01563594  55.9037397   66.04565971
  87.30605557  91.48539457  98.70716945 112.3882742  119.14908906
 144.47751994 157.75788857 286.12227087]
Mean cost:  80.47575738955175
Median cost:  60.97469970728498
S

Median cost:  96.8746725885222
Std cost:  73.97333991629591

List:  list_487
Cost values:  [1.e+12]
Mean cost:  1000000000000.0
Median cost:  1000000000000.0
Std cost:  0.0

List:  list_488
Cost values:  [ 15.29341932 157.17698077 357.54846193]
Mean cost:  176.6729540061864
Median cost:  157.17698076805075
Std cost:  140.4034621541644

List:  list_489
Cost values:  [ 27.572124    29.21278213  38.01784394  47.63719102 149.86719172]
Mean cost:  58.461426561666414
Median cost:  38.01784393670764
Std cost:  46.259594167106705

List:  list_490
Cost values:  [ 59.22710715 109.68479106]
Mean cost:  84.45594910561807
Median cost:  84.45594910561807
Std cost:  25.22884195659107

List:  list_491
Cost values:  [ 16.47785829  98.43190143 154.74817957 403.02770779 405.22737361
 493.72020581]
Mean cost:  261.9388710844923
Median cost:  278.887943680705
Std cost:  179.18134398886264

List:  list_492
Cost values:  [  8.4916762   11.17888931  17.36445145  29.10372638  45.57514052
  85.08263056 108.0052

Cost values:  [73.39751446]
Mean cost:  73.39751446464271
Median cost:  73.39751446464271
Std cost:  0.0

List:  list_709
Cost values:  [52.29204026 85.1281983 ]
Mean cost:  68.71011927965776
Median cost:  68.71011927965776
Std cost:  16.418079023763163

List:  list_710
Cost values:  [ 47.42994861 101.33811533 171.82527294 226.35433217 565.37319373]
Mean cost:  222.46417255432956
Median cost:  171.8252729402506
Std cost:  181.92005364423704

List:  list_711
Cost values:  [ 15.58051074  46.35560665  78.38447285 106.50843233 125.05408689
 370.25090806 515.84664491 592.346838  ]
Mean cost:  231.2909375551688
Median cost:  115.78125961049543
Std cost:  212.60661475096987

List:  list_712
Cost values:  [  17.76407234   33.52890761   35.68399879   93.35351727 1812.46266499]
Mean cost:  398.5586321998981
Median cost:  35.68399878798736
Std cost:  707.4184012819369

List:  list_713
Cost values:  [ 67.726528   246.56438189 827.08546055]
Mean cost:  380.45879014862044
Median cost:  246.564381890

Median cost:  824.5225263627058
Std cost:  706.6394825098328

List:  list_937
Cost values:  [  25.43708145   89.38851915   90.83724889  308.83914743 2674.67384661]
Mean cost:  637.8351687045008
Median cost:  90.83724889149595
Std cost:  1022.9351935268735

List:  list_938
Cost values:  [238.99861615]
Mean cost:  238.99861614645408
Median cost:  238.99861614645408
Std cost:  0.0

List:  list_939
Cost values:  [427.00595432]
Mean cost:  427.00595432240823
Median cost:  427.00595432240823
Std cost:  0.0

List:  list_940
Cost values:  [103.60618586 687.12085928]
Mean cost:  395.363522568845
Median cost:  395.363522568845
Std cost:  291.75733671288805

List:  list_941
Cost values:  [59.31365586]
Mean cost:  59.31365585983252
Median cost:  59.31365585983252
Std cost:  0.0

List:  list_942
Cost values:  [174.09514565 185.40661269 247.22788059 351.13577525 486.29996727
 556.7838902  675.26439294]
Mean cost:  382.31623779935484
Median cost:  351.1357752528988
Std cost:  180.6419934192332

List:

Median cost:  68.74598466795749
Std cost:  51.707343767540024

List:  list_1165
Cost values:  [88.22815966]
Mean cost:  88.2281596632539
Median cost:  88.2281596632539
Std cost:  0.0

List:  list_1166
Cost values:  [990.18812896]
Mean cost:  990.1881289587244
Median cost:  990.1881289587244
Std cost:  0.0

List:  list_1167
Cost values:  [ 51.29537805 174.30741553 185.48373833]
Mean cost:  137.02884396770898
Median cost:  174.30741552850026
Std cost:  60.79417698024444

List:  list_1168
Cost values:  [  7.75586648  85.65918404  86.18014612 128.40723233 187.93286933
 220.27468341]
Mean cost:  119.36833028494705
Median cost:  107.29368922466782
Std cost:  70.3088354446903

List:  list_1169
Cost values:  [ 65.53984829 251.16181405]
Mean cost:  158.35083116814403
Median cost:  158.35083116814403
Std cost:  92.810982878525

List:  list_1170
Cost values:  [ 42.64964872 307.92474753 449.31264284]
Mean cost:  266.6290130273435
Median cost:  307.9247475253934
Std cost:  168.5678912955129

List: 

Mean cost:  14.006831872612993
Median cost:  14.006831872612993
Std cost:  0.0

List:  list_1387
Cost values:  [ 6.12382772 18.09688901]
Mean cost:  12.110358364917534
Median cost:  12.110358364917534
Std cost:  5.9865306483503185

List:  list_1388
Cost values:  [7.87178450e+01 8.96986526e+02 1.00000000e+12]
Mean cost:  333333333658.5681
Median cost:  896.9865262482302
Std cost:  471404520561.05597

List:  list_1389
Cost values:  [7147.93277075]
Mean cost:  7147.932770753078
Median cost:  7147.932770753078
Std cost:  0.0

List:  list_1390
Cost values:  [629.05953025]
Mean cost:  629.0595302534917
Median cost:  629.0595302534917
Std cost:  0.0

List:  list_1391
Cost values:  [ 82.40597836 687.05848591]
Mean cost:  384.7322321328872
Median cost:  384.7322321328872
Std cost:  302.3262537735862

List:  list_1392
Cost values:  [189.23936512 203.17629719 327.85448257 347.94222305 364.2576192 ]
Mean cost:  286.49399742485554
Median cost:  327.8544825673823
Std cost:  74.7449767032838

List:  

In [41]:
print_lim = 100
for k in range(N_list):
    list_name = "list_" + str(k+1)
    algorithm_count = [0,0,0,0,0]
    
    if data_list_no_failures[list_name]:
        arr = np.array(data_list_no_failures[list_name],dtype=np.float)
        print "\nList: ", list_name
        print "Method - Cost"
        for i, (method, cost) in enumerate(zip(arr[:,16],arr[:,17])):
            if i < 10:
                print algorithms[int(method)] + " - " + str(cost)
            
            if i < print_lim:
                algorithm_count[int(method)] += 1.
            elif i == print_lim:
                print "Limit Cost: ", cost
        
#         print "\nCost values: ", arr[:,17]
        print "\nMean cost: ", np.mean(arr[:,17])
        print "Median cost: ", np.median(arr[:,17])
        print "Std cost: ", np.std(arr[:,17])
        
        print "\nDistribution on the first {0} data points".format(print_lim)
        print "N_min_vel: {0:.3}".format(algorithm_count[0] / np.sum(algorithm_count))
        print "N_min_acc: {0:.3}".format(algorithm_count[1] / np.sum(algorithm_count))
        print "N_min_jerk: {0:.3}".format(algorithm_count[2] / np.sum(algorithm_count)) 
        print "N_min_jerk_stop: {0:.3}".format(algorithm_count[3] / np.sum(algorithm_count)) 


List:  list_1
Method - Cost
min_vel - 2.866649783177905
min_jerk_full_stop - 3.5134219235339486
min_jerk - 4.204343677225984
min_jerk_full_stop - 4.843232878998881
min_acc - 5.591031103580179
min_jerk - 5.7968272873004585
min_acc - 6.248548434153585
min_jerk - 6.978616963204867
min_acc - 7.041200628644763
min_jerk_full_stop - 7.985864564006411
Limit Cost:  43.513127428920185

Mean cost:  4001.3713387148955
Median cost:  619.6702919722766
Std cost:  8834.343338380108

Distribution on the first 100 data points
N_min_vel: 0.12
N_min_acc: 0.23
N_min_jerk: 0.25
N_min_jerk_stop: 0.4

List:  list_2
Method - Cost
min_jerk_full_stop - 12.485988924825449
min_jerk_full_stop - 20.052403332813608
min_jerk_full_stop - 21.657937229676307
min_acc - 23.182804326413613
min_jerk - 23.69935769491601
min_jerk - 25.308831462365127
min_acc - 27.705467430413552
min_jerk_full_stop - 32.045670957035945
min_acc - 37.28829002254741
min_acc - 41.0325060266768
Limit Cost:  587.3258532365019

Mean cost:  341.094571

Method - Cost
min_jerk - 85.5134760175149

Mean cost:  85.5134760175149
Median cost:  85.5134760175149
Std cost:  0.0

Distribution on the first 100 data points
N_min_vel: 0.0
N_min_acc: 0.0
N_min_jerk: 1.0
N_min_jerk_stop: 0.0

List:  list_161
Method - Cost
min_jerk_full_stop - 368.7698119041341
min_vel - 1143.496515045561
min_vel - 1537.7833617760293
min_jerk_full_stop - 1640.4562940235226
min_vel - 1710.8935043124802
min_jerk_full_stop - 1718.826659610553
min_jerk_full_stop - 1926.5747852771808
min_vel - 2022.655152105856
min_jerk - 5468.100356405514
min_jerk - 5610.1273255866245

Mean cost:  2639.095665596198
Median cost:  1718.826659610553
Std cost:  1896.0873732496573

Distribution on the first 100 data points
N_min_vel: 0.364
N_min_acc: 0.0
N_min_jerk: 0.273
N_min_jerk_stop: 0.364

List:  list_162
Method - Cost
min_jerk_full_stop - 23.337453319770304
min_jerk_full_stop - 41.68200138588934
min_jerk - 247.1236112950599
min_vel - 434.6710688940634
min_acc - 616.1801919994167
min_je

N_min_vel: 0.0
N_min_acc: 0.0
N_min_jerk: 1.0
N_min_jerk_stop: 0.0

List:  list_330
Method - Cost
min_jerk - 14.592037668590065
min_jerk - 53.867653424953616
min_jerk - 68.89685870609024
min_jerk - 69.66937380783222
min_jerk - 104.45147784143488
min_vel - 154.66718094666692
min_jerk - 165.48097023338158
min_jerk - 179.86158946931096
min_jerk_full_stop - 183.8013504380404
min_jerk - 219.339576813014

Mean cost:  165.92180054741908
Median cost:  172.67127985134627
Std cost:  94.68011399031678

Distribution on the first 100 data points
N_min_vel: 0.143
N_min_acc: 0.0
N_min_jerk: 0.786
N_min_jerk_stop: 0.0714

List:  list_331
Method - Cost
min_acc - 45.941800794079285
min_jerk - 51.754253572009105
min_vel - 65.47281134366861
min_jerk_full_stop - 74.46266401447492
min_jerk_full_stop - 123.28406291950128
min_acc - 148.41854760353826
min_vel - 206.84482894502716
min_vel - 302.8277102254074
min_acc - 379.6569857105088

Mean cost:  155.40707390313497
Median cost:  123.28406291950128
Std cost:  

min_acc - 174.89238601200054

Mean cost:  123.82686188395996
Median cost:  105.54207991048443
Std cost:  36.59051818205142

Distribution on the first 100 data points
N_min_vel: 0.333
N_min_acc: 0.333
N_min_jerk: 0.333
N_min_jerk_stop: 0.0

List:  list_537
Method - Cost
min_vel - 12.136028938283276

Mean cost:  12.136028938283276
Median cost:  12.136028938283276
Std cost:  0.0

Distribution on the first 100 data points
N_min_vel: 1.0
N_min_acc: 0.0
N_min_jerk: 0.0
N_min_jerk_stop: 0.0

List:  list_538
Method - Cost
min_acc - 20.206068170664352
min_vel - 136.04601542275321

Mean cost:  78.12604179670879
Median cost:  78.12604179670879
Std cost:  57.91997362604443

Distribution on the first 100 data points
N_min_vel: 0.5
N_min_acc: 0.5
N_min_jerk: 0.0
N_min_jerk_stop: 0.0

List:  list_539
Method - Cost
min_jerk - 35.919727379340515
min_jerk - 40.075622250201505
min_jerk_full_stop - 42.29846612541141

Mean cost:  39.43127191831781
Median cost:  40.075622250201505
Std cost:  2.6436675926075

N_min_acc: 0.0
N_min_jerk: 0.375
N_min_jerk_stop: 0.0

List:  list_767
Method - Cost
min_jerk - 52.21708311031657
min_jerk - 64.71050342877572
min_vel - 91.7516177390742
min_jerk - 154.33552634411697
min_jerk - 214.7444162236729
min_jerk - 218.68910890951776

Mean cost:  132.7413759592457
Median cost:  123.04357204159558
Std cost:  67.54053150734781

Distribution on the first 100 data points
N_min_vel: 0.167
N_min_acc: 0.0
N_min_jerk: 0.833
N_min_jerk_stop: 0.0

List:  list_769
Method - Cost
min_vel - 306.88508619784784
min_vel - 594.299454167992

Mean cost:  450.59227018291995
Median cost:  450.59227018291995
Std cost:  143.7071839850721

Distribution on the first 100 data points
N_min_vel: 1.0
N_min_acc: 0.0
N_min_jerk: 0.0
N_min_jerk_stop: 0.0

List:  list_770
Method - Cost
min_acc - 16.919034482916622
min_acc - 41.33629599040604
min_vel - 55.5624133525407
min_vel - 62.028785471536786
min_jerk - 63.06670679404258
min_acc - 97.26265530368627
min_vel - 246.71807719928336

Mean cost:  

N_min_acc: 0.0
N_min_jerk: 0.0
N_min_jerk_stop: 0.0

List:  list_1049
Method - Cost
min_acc - 40.7727446648591
min_vel - 110.76819788196572
min_vel - 141.1433683707582
min_vel - 243.69260588941685

Mean cost:  134.09422920174998
Median cost:  125.95578312636195
Std cost:  72.99742760689668

Distribution on the first 100 data points
N_min_vel: 0.75
N_min_acc: 0.25
N_min_jerk: 0.0
N_min_jerk_stop: 0.0

List:  list_1050
Method - Cost
min_vel - 40.0660278840361
min_vel - 61.851134581820794
min_vel - 120.96746794605343
min_vel - 155.7199168017838

Mean cost:  94.65113680342353
Median cost:  91.40930126393711
Std cost:  46.03617989357722

Distribution on the first 100 data points
N_min_vel: 1.0
N_min_acc: 0.0
N_min_jerk: 0.0
N_min_jerk_stop: 0.0

List:  list_1051
Method - Cost
min_vel - 1413.8226230163245

Mean cost:  1413.8226230163245
Median cost:  1413.8226230163245
Std cost:  0.0

Distribution on the first 100 data points
N_min_vel: 1.0
N_min_acc: 0.0
N_min_jerk: 0.0
N_min_jerk_stop: 0.0

N_min_vel: 0.25
N_min_acc: 0.0
N_min_jerk: 0.75
N_min_jerk_stop: 0.0

List:  list_1425
Method - Cost
min_jerk_full_stop - 34.044940345310195
min_jerk - 200.05829007586397

Mean cost:  117.05161521058707
Median cost:  117.05161521058707
Std cost:  83.00667486527688

Distribution on the first 100 data points
N_min_vel: 0.0
N_min_acc: 0.0
N_min_jerk: 0.5
N_min_jerk_stop: 0.5

List:  list_1427
Method - Cost
min_acc - 81.27437354207349

Mean cost:  81.27437354207349
Median cost:  81.27437354207349
Std cost:  0.0

Distribution on the first 100 data points
N_min_vel: 0.0
N_min_acc: 1.0
N_min_jerk: 0.0
N_min_jerk_stop: 0.0

List:  list_1431
Method - Cost
min_jerk - 77.22188629364902

Mean cost:  77.22188629364902
Median cost:  77.22188629364902
Std cost:  0.0

Distribution on the first 100 data points
N_min_vel: 0.0
N_min_acc: 0.0
N_min_jerk: 1.0
N_min_jerk_stop: 0.0

List:  list_1433
Method - Cost
min_vel - 31.70264653465372
min_vel - 77.57634718550422

Mean cost:  54.63949686007897
Median co

### Dataset Preparation

In [50]:
cost_upper_lim = 1e9

method_list = ["min_vel", "min_acc", "min_jerk", "min_jerk_full_stop"]

mp_classifier_data = []
time_regressor_data = []

for k in range(N_list):
    list_name = "list_" + str(k+1)
    time_data_appending = False
    if data_list[list_name]:
        print "\nList: ", list_name
        arr = np.array(data_list[list_name],dtype=np.float32).reshape(-1, 18)
        best_method = int(arr[0][16])
        for i in range(len(data_list[list_name])):
            cost_val = arr[i][17]
            # check if cost of each data point is less than mean cost and upper limit of cost value
            if cost_val <= cost_median[list_name] and cost_val < cost_upper_lim: 
                
#                 print "Data point, dx: {0:.4}, dy: {1:.4}, dz: {2:.4}, Tf: {3:.4}, v_avg: {4:.4}, cost:{5:.4}, method: {6}, type: {7}".format(arr[i][3],arr[i][4],arr[i][5],
#                                                                                                                                    arr[i][14],arr[i][15], arr[i][17], 
#                                                                                                                                    method_list[int(arr[i][16])],type_list[int(arr[i][13])])
                #if the output of current data point is consistent with the output of the data point, which has the lowest cost
                #then add it into the dataset (they should not conflict each other)
                if int(arr[i][16]) == best_method:
                    #true_init_x, true_init_y, true_init_z, blur_coeff, var_sum, diff_x, diff_y, diff_z, diff_phi, diff_theta, diff_psi, r_std, phi_std, theta_std, psi_std, Tf, MP_Method, Cost
                    mp_classifier_data.append([arr[i][5],arr[i][6],arr[i][7],arr[i][8],arr[i][9],arr[i][10], 
                                          arr[i][4],arr[i][11],arr[i][12],arr[i][13],arr[i][14],int(arr[i][16])])
                    print "MP Classifier, dx: {0:.4}, dy: {1:.4}, dz: {2:.4}, cost:{3:.4}, method: {4}".format(arr[i][5],arr[i][6],arr[i][7],
                                                                                                            arr[i][17], method_list[int(arr[i][16])])
                
                #If data point belongs time based algorithm
                #then add it into the time_regressor dataset
                if not time_data_appending: 
                    #true_init_x, true_init_y, true_init_z, blur_coeff, var_sum, diff_x, diff_y, diff_z, diff_phi, diff_theta, diff_psi, r_std, phi_std, theta_std, psi_std, Tf, MP_Method, Cost
                    time_regressor_data.append([arr[i][5],arr[i][6],arr[i][7],arr[i][8],arr[i][9],arr[i][10], 
                                          arr[i][4],arr[i][11],arr[i][12],arr[i][13],arr[i][14],arr[i][15]])
                    
                    time_data_appending = True
                    print "Time Regressor, dx: {0:.4}, dy: {1:.4}, dz: {2:.4}, cost:{3:.4}, Tf: {4:.4}".format(arr[i][5],arr[i][6],arr[i][7],
                                                                                                    arr[i][17], arr[i][15])
            elif cost_val > cost_upper_lim: #failed flights should be in dataset as well
                #true_init_x, true_init_y, true_init_z, blur_coeff, var_sum, diff_x, diff_y, diff_z, diff_phi, diff_theta, diff_psi, r_std, phi_std, theta_std, psi_std, Tf, MP_Method, Cost
                mp_classifier_data.append([arr[i][5],arr[i][6],arr[i][7],arr[i][8],arr[i][9],arr[i][10], 
                                          arr[i][4],arr[i][11],arr[i][12],arr[i][13],arr[i][14],4])
            
mp_classifier_data = np.array(mp_classifier_data)
time_regressor_data = np.array(time_regressor_data)

print ""
print "mp_classifier_data shape: ", mp_classifier_data.shape
print "time_regressor_data shape: ", time_regressor_data.shape


List:  list_1
MP Classifier, dx: 0.2524, dy: -3.577, dz: 0.2004, cost:542.2, method: min_vel
Time Regressor, dx: 0.2524, dy: -3.577, dz: 0.2004, cost:542.2, Tf: 2.53
MP Classifier, dx: 0.2341, dy: -3.524, dz: 0.4184, cost:597.0, method: min_vel
MP Classifier, dx: -0.06851, dy: -3.518, dz: -0.2731, cost:750.5, method: min_vel
MP Classifier, dx: 0.1984, dy: -3.487, dz: 0.4164, cost:951.3, method: min_vel
MP Classifier, dx: 0.05051, dy: -3.662, dz: -0.3812, cost:1.009e+03, method: min_vel
MP Classifier, dx: -0.1276, dy: -3.578, dz: -0.2199, cost:1.087e+03, method: min_vel

List:  list_2
MP Classifier, dx: 0.1513, dy: -2.767, dz: -1.057, cost:151.7, method: min_jerk_full_stop
Time Regressor, dx: 0.1513, dy: -2.767, dz: -1.057, cost:151.7, Tf: 1.904
MP Classifier, dx: 0.05416, dy: -2.987, dz: -0.5238, cost:165.6, method: min_jerk_full_stop
MP Classifier, dx: 0.3099, dy: -3.148, dz: -0.7137, cost:206.4, method: min_jerk_full_stop
MP Classifier, dx: 0.01995, dy: -2.943, dz: -0.9014, cost:210

List:  list_569
MP Classifier, dx: 0.3491, dy: -0.8031, dz: -0.8062, cost:680.4, method: min_jerk
Time Regressor, dx: 0.3491, dy: -0.8031, dz: -0.8062, cost:680.4, Tf: 0.8174

List:  list_570
MP Classifier, dx: 1.248, dy: -2.51, dz: -0.2395, cost:910.1, method: min_jerk
Time Regressor, dx: 1.248, dy: -2.51, dz: -0.2395, cost:910.1, Tf: 1.796

List:  list_571
MP Classifier, dx: 0.7625, dy: -1.606, dz: -0.8646, cost:925.5, method: min_jerk
Time Regressor, dx: 0.7625, dy: -1.606, dz: -0.8646, cost:925.5, Tf: 1.291

List:  list_572
MP Classifier, dx: 0.3871, dy: -1.268, dz: -0.1796, cost:1.063e+03, method: min_jerk
Time Regressor, dx: 0.3871, dy: -1.268, dz: -0.1796, cost:1.063e+03, Tf: 0.9061

List:  list_573

List:  list_574
MP Classifier, dx: -0.4016, dy: -1.025, dz: 0.8042, cost:1.835e+03, method: min_jerk
Time Regressor, dx: -0.4016, dy: -1.025, dz: 0.8042, cost:1.835e+03, Tf: 0.9218

List:  list_575

List:  list_576

List:  list_577
MP Classifier, dx: 0.07953, dy: -1.648, dz: 0.6861,

### Dataset Analysis

In [52]:
class_labels = {"min_vel":0, "min_acc":1, "min_jerk":2, "min_jerk_full_stop":3, "safe_mode":4}
N_length = float(mp_classifier_data.shape[0])
print "MP Dataset size: ",mp_classifier_data.shape 
print "N_min_vel: ",np.sum(mp_classifier_data[:,-1] == class_labels["min_vel"]) / N_length * 100
print "N_min_acc: ",np.sum(mp_classifier_data[:,-1] == class_labels["min_acc"]) / N_length * 100
print "N_min_jerk: ",np.sum(mp_classifier_data[:,-1] == class_labels["min_jerk"]) / N_length * 100
print "N_min_jerk_stop: ",np.sum(mp_classifier_data[:,-1] == class_labels["min_jerk_full_stop"]) / N_length * 100
print "N_safe_mode: ",np.sum(mp_classifier_data[:,-1] == class_labels["safe_mode"]) / N_length * 100

MP Dataset size:  (1327, 12)
N_min_vel:  10.399397136397889
N_min_acc:  15.749811605124343
N_min_jerk:  15.900527505651846
N_min_jerk_stop:  57.573474001507165
N_safe_mode:  0.37678975131876413


### Split of dataset as train, validation and test

In [60]:
MP_Test = 200
Time_Test = 75
np.random.shuffle(mp_classifier_data) # Train 1
np.random.shuffle(time_regressor_data) # Train 2

y1 = np.array(mp_classifier_data[:,-1],dtype=np.int)
y2 = np.array(time_regressor_data[:,-1],dtype=np.float)
X1 = mp_classifier_data[:,:-1]
X2 = time_regressor_data[:,:-1]

X1_test = X1[0:MP_Test,:]
y1_test = y1[0:MP_Test]
X2_test = X2[0:Time_Test,:]
y2_test = y2[0:Time_Test]

X1_train, X1_val, y1_train, y1_val = train_test_split(X1[MP_Test:,:], y1[MP_Test:], test_size=0.2, random_state=42)
X2_train, X2_val, y2_train, y2_val = train_test_split(X2[Time_Test:,:], y2[Time_Test:], test_size=0.2, random_state=42)
    
pickle.dump([X1_train, X1_val, X1_test, y1_train, y1_val, y1_test], open("classifier_files/dataset1.pkl","wb"), protocol=2)
pickle.dump([X2_train, X2_val, X2_test, y2_train, y2_val, y2_test], open("classifier_files/dataset2.pkl","wb"), protocol=2)

scaler1,scaler2 = StandardScaler(), StandardScaler()
scaler1.fit(X1_train)
scaler2.fit(X2_train)
X1_train, X2_train = scaler1.transform(X1_train), scaler2.transform(X2_train)
X1_val, X2_val = scaler1.transform(X1_val), scaler2.transform(X2_val)
X1_test, X2_test = scaler1.transform(X1_test), scaler2.transform(X2_test)

dump(scaler1, 'classifier_files/mp_scaler.bin', compress=True)
dump(scaler2, 'classifier_files/time_scaler.bin', compress=True)

print "X1_train: " + str(X1_train.shape) + " X2_train: " + str(X2_train.shape)
print "X1_val: " + str(X1_val.shape) + " X2_val: " + str(X2_val.shape) 
print "X1_test: " + str(X1_test.shape) + " X2_test: " + str(X2_test.shape) 

X1_train: (901, 11) X2_train: (310, 11)
X1_val: (226, 11) X2_val: (78, 11)
X1_test: (200, 11) X2_test: (75, 11)


### Train and predict functions

In [55]:
def predict(X, y, model, criterion, isClassifier):
    #Validation part
    model.eval()  # Set model to training mode
    
    inputs = torch.from_numpy(X).to(device)
    if isClassifier:
        labels = torch.from_numpy(y).to(device).long()
    else:
        labels = torch.from_numpy(y).to(device).float()

    outputs = model(inputs.float())
    if isClassifier:
        _, preds = torch.max(outputs, 1)

    loss = criterion(outputs, labels)
    
    if isClassifier:
        accuracy = torch.sum(preds == labels.data).item() / float(inputs.size(0))
        print "Test data, Loss: {0:.3}, Accuracy: {1:.4}".format(loss.item(), accuracy)
    else:
        print "Test data, Loss: {0:.3}\n".format(loss.item())
        print "Actual - Predicted"
        pred = outputs.detach().cpu().numpy()
        for i in range(y.shape[0]):
            print "{0:.3} - {1:.3}".format(y[i], outputs[i].item())
        
    

def shuffle_dataset(X, y):
    p = np.random.permutation(len(X))
    return X[p], y[p]

def train_model(isClassifier, X, y, X_val, y_val, model, criterion, optimizer, scheduler, minibatch_size, name, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_train_acc = 0.0
    best_acc = 0.0
    best_loss = 1e6
    best_train_loss = 1e6
    
    losses_train = []
    losses_val = []
    accuracy_train = []
    accuracy_val = []
    # path = F"/content/drive/My Drive/best_model.pt"
#     directory = path_name

#     if not os.path.exists(directory):
#         os.makedirs(directory)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        X_train, y_train = shuffle_dataset(X, y)

        # Each epoch has a training and validation phase
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0
        losses_iter = []
        accuracy_iter = []

        # Iterate over data.
        for i in range(0, X_train.shape[0], minibatch_size):
            # Get pair of (X, y) of the current minibatch/chunk             
            X_batch = X_train[i:i + minibatch_size]
            y_batch = y_train[i:i + minibatch_size]

            # get the inputs; data is a list of [inputs, labels]
            xbatch = torch.from_numpy(X_batch).to(device)
            
            if isClassifier:
                ybatch = torch.from_numpy(y_batch).to(device).long()
            else:
                ybatch = torch.from_numpy(y_batch).to(device).float()


            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                pred = model(xbatch.float())
                if isClassifier:
                    _, preds = torch.max(pred, 1)
                
                loss = criterion(pred, ybatch)

                # backward + optimize only if in training phase
                loss.backward()
                optimizer.step()
            
            
            # statistics
            running_loss += loss.item() * xbatch.size(0)
            if isClassifier:
                running_corrects += torch.sum(preds == ybatch.data)

            # print ("losses_iter", loss.item() * inputs.size(0))
            # print ("accuracy_iter", torch.sum(preds == labels.data).item() / float(inputs.size(0)))

            losses_iter.append(loss.item())
            if isClassifier:
                accuracy_iter.append(torch.sum(preds == ybatch.data).item() / float(xbatch.size(0)))
        
        
        train_loss = np.mean(losses_iter)
        losses_train.append(train_loss)
        if isClassifier:
            train_acc = np.mean(accuracy_iter)
            accuracy_train.append(train_acc)
            print'Training Loss: {:.4f} Acc: {:.4f}'.format(train_loss, train_acc)  
        else:
            print'Training Loss: {:.4f}'.format(train_loss)
 
        
        
        #Validation part
        model.eval()  # Set model to training mode
        
        xbatch = torch.from_numpy(X_val).to(device)
        if isClassifier:
            ybatch = torch.from_numpy(y_val).to(device).long()
        else:
            ybatch = torch.from_numpy(y_val).to(device).float()
        
        optimizer.zero_grad()
        
        pred = model(xbatch.float())
        if isClassifier:
            _, preds = torch.max(pred, 1)

        loss = criterion(pred, ybatch)
        
        val_loss = loss.item()
        scheduler.step(val_loss)
        
        if isClassifier:
            val_acc = torch.sum(preds == ybatch.data).item() / float(xbatch.size(0))
            accuracy_val.append(val_acc)
        
        losses_val.append(val_loss)
        
        
        if isClassifier:
            print'Validation Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc)
        else:
            print'Validation Loss: {:.4f}'.format(val_loss)

#         deep copy the model
        if isClassifier:
            if val_acc > best_acc:
                best_train_acc = train_acc
                best_acc = val_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                path = name + "_best_model.pt"
                torch.save(best_model_wts, "classifier_files/" + path)
        else:
            if val_loss < best_loss:
                best_train_loss = train_loss
                best_loss = val_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                path = name + "_best_model.pt"
                torch.save(best_model_wts, "classifier_files/" + path)

        print ""


    time_elapsed = time.time() - since
    print'Training complete in {:.0f}s'.format(time_elapsed)
    if isClassifier:
        print'Best Train Acc: {:4f}'.format(best_train_acc)
        print'Best Val Acc: {:4f}'.format(best_acc)
    else:
        print'Best Train Loss: {:4f}'.format(best_train_loss)
        print'Best Val Loss: {:4f}'.format(best_loss)

    
#     stats_columns = ['Layers', 'Epochs', 'BatchSize', 'LearningRate', 'Optimizer', 'Scheduler', 'TrainAcc', 'ValAcc']
#     layers = [module for module in model.modules() if type(module) != nn.Sequential]
    #write_results([layers, num_epochs, minibatch_size, learning_rate, optimizer.state_dict, scheduler.state_dict, best_train_acc, best_val_acc])

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model

## MP Classifier Training

In [56]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop_layer = nn.Dropout(p=0.0)
        self.fc1 = nn.Linear(11, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc2(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

n_epochs = 30
minibatch_size = 16
learning_rate = 0.001
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = Net()
model = model.to(device)
ce_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = lr_scheduler.CosineAnnealingLR(optimizer, X_train.shape[0], eta_min=learning_rate)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
mp_model = train_model(True, X1_train, y1_train, X1_val, y1_val, model, ce_criterion, optimizer, scheduler, minibatch_size, "mp_classifier", num_epochs=n_epochs)

Epoch 0/29
----------
Training Loss: 1.3408 Acc: 0.5388
Validation Loss: 1.1678 Acc: 0.5841

Epoch 1/29
----------
Training Loss: 1.1129 Acc: 0.6079
Validation Loss: 1.0609 Acc: 0.6283

Epoch 2/29
----------
Training Loss: 1.0501 Acc: 0.6235
Validation Loss: 1.0150 Acc: 0.6239

Epoch 3/29
----------
Training Loss: 1.0405 Acc: 0.6197
Validation Loss: 0.9983 Acc: 0.6283

Epoch 4/29
----------
Training Loss: 1.0111 Acc: 0.6292
Validation Loss: 0.9911 Acc: 0.6283

Epoch 5/29
----------
Training Loss: 1.0058 Acc: 0.6320
Validation Loss: 0.9705 Acc: 0.6504

Epoch 6/29
----------
Training Loss: 0.9930 Acc: 0.6362
Validation Loss: 0.9554 Acc: 0.6504

Epoch 7/29
----------
Training Loss: 0.9744 Acc: 0.6498
Validation Loss: 0.9406 Acc: 0.6593

Epoch 8/29
----------
Training Loss: 0.9601 Acc: 0.6498
Validation Loss: 0.9319 Acc: 0.6593

Epoch 9/29
----------
Training Loss: 0.9481 Acc: 0.6511
Validation Loss: 0.9110 Acc: 0.6549

Epoch 10/29
----------
Training Loss: 0.9386 Acc: 0.6509
Validation Lo

In [59]:
print "Test Results for MP Classification"
predict(X1_test, y1_test, mp_model, ce_criterion, True)

Test Results for MP Classification
Test data, Loss: 0.931, Accuracy: 0.665


## Time Regressor Training

In [57]:
class Net_Regressor(nn.Module):
    def __init__(self):
        super(Net_Regressor, self).__init__()
        self.drop_layer = nn.Dropout(p=0.15)
        self.fc1 = nn.Linear(11, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc2(x))
        x = self.drop_layer(x)
        x = self.fc3(x)
        return x
    
n_epochs = 30
minibatch_size = 16
learning_rate = 0.001
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = Net_Regressor()
model = model.to(device)
mse_criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = lr_scheduler.CosineAnnealingLR(optimizer, X_train.shape[0], eta_min=learning_rate)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
time_model = train_model(False, X2_train, y2_train, X2_val, y2_val, model, mse_criterion, optimizer, scheduler, minibatch_size, 'time_regressor', num_epochs=n_epochs)

Epoch 0/29
----------
Training Loss: 2.4602
Validation Loss: 1.6790

Epoch 1/29
----------
Training Loss: 1.0479
Validation Loss: 0.7556

Epoch 2/29
----------
Training Loss: 0.8396
Validation Loss: 0.6843

Epoch 3/29
----------
Training Loss: 0.7531
Validation Loss: 0.6565

Epoch 4/29
----------
Training Loss: 0.7147
Validation Loss: 0.6238

Epoch 5/29
----------
Training Loss: 0.7215
Validation Loss: 0.6196

Epoch 6/29
----------
Training Loss: 0.7047
Validation Loss: 0.6188

Epoch 7/29
----------
Training Loss: 0.7039
Validation Loss: 0.6145

Epoch 8/29
----------
Training Loss: 0.7150
Validation Loss: 0.6058

Epoch 9/29
----------
Training Loss: 0.7060
Validation Loss: 0.6007

Epoch 10/29
----------
Training Loss: 0.6747
Validation Loss: 0.6010

Epoch 11/29
----------
Training Loss: 0.6855
Validation Loss: 0.5997

Epoch 12/29
----------
Training Loss: 0.6830
Validation Loss: 0.5945

Epoch 13/29
----------
Training Loss: 0.6543
Validation Loss: 0.5791

Epoch 14/29
----------
Trainin

In [58]:
print "Test Results for Time Regression"
predict(X2_test, y2_test, time_model, mse_criterion, False)

Test Results for Time Regression
Test data, Loss: 0.579

Actual - Predicted
2.54 - 1.64
1.16 - 1.5
2.61 - 1.62
1.48 - 1.65
2.32 - 1.83
3.29 - 1.65
2.88 - 1.43
1.88 - 1.48
0.292 - 1.52
2.51 - 1.64
2.17 - 1.86
2.64 - 1.76
1.27 - 1.4
2.6 - 1.32
1.64 - 1.53
1.36 - 1.53
1.7 - 1.68
0.305 - 1.52
1.48 - 1.38
1.8 - 1.43
1.7 - 1.58
1.63 - 1.52
1.84 - 1.67
1.47 - 1.56
2.1 - 1.88
1.25 - 1.55
1.42 - 1.43
0.456 - 1.62
0.934 - 1.38
1.35 - 1.51
3.71 - 1.63
1.52 - 1.44
1.05 - 1.48
0.739 - 1.74
1.33 - 1.54
1.82 - 1.69
2.47 - 1.72
1.05 - 1.49
1.24 - 1.53
1.08 - 1.4
1.34 - 1.62
2.01 - 1.52
2.11 - 1.93
1.22 - 1.59
1.93 - 1.57
1.68 - 1.43
1.41 - 1.37
1.76 - 1.57
0.645 - 1.49
3.01 - 1.65
2.87 - 1.79
2.34 - 1.79
1.09 - 1.67
1.85 - 1.65
2.55 - 1.75
2.42 - 1.72
2.11 - 1.58
1.49 - 1.4
0.968 - 1.57
2.33 - 1.7
1.25 - 1.45
0.753 - 1.36
1.28 - 1.67
1.41 - 1.49
2.96 - 1.81
0.263 - 1.56
2.35 - 1.63
0.994 - 1.65
0.294 - 1.51
2.03 - 1.64
1.49 - 1.59
1.33 - 1.56
0.943 - 1.48
0.812 - 1.8
2.58 - 1.73
